In [ ]:
import pandas as pd
file_path = "/home/sagemaker-user/TrajCL/data/porto_20200"
trajs = pd.read_pickle(file_path)

In [ ]:
trajs.head()

In [ ]:
# from torch.utils.data import Dataset
# class TrajDataset(Dataset):
#     def __init__(self, data):
#         # data: DataFrame
#         self.data = data

#     def __getitem__(self, index):
#         return self.data.loc[index].merc_seq

#     def __len__(self):
#         return self.data.shape[0]

In [ ]:
# l = trajs.shape[0]
# train_idx = (int(l*0), 200000)
# eval_idx = (int(l*0.7), int(l*0.8))
# test_idx = (int(l*0.8), int(l*1.0))

# train = TrajDataset(trajs[train_idx[0]: train_idx[1]])
# eval = TrajDataset(trajs[eval_idx[0]: eval_idx[1]])
# test = TrajDataset(trajs[test_idx[0]: test_idx[1]])

In [ ]:
idx = 15
lon_lat_list = trajs['wgs_seq'][idx]

In [ ]:
lon_lat_list

In [ ]:
# !pip install folium
import folium

from folium import plugins
coordinates = [[x[0],x[1]] for x in lon_lat_list]
# Initialize map
m = folium.Map(location=coordinates[0], zoom_start=30)

folium.Marker(
    location=coordinates[0],
    popup="Start Location",
    icon=folium.Icon(color='green')  # Color can be 'red', 'blue', 'green', 'purple', etc.
).add_to(m)
# Add markers
for lat, lon in coordinates[1:]:
    folium.Marker([lat, lon]).add_to(m)

# Draw arrows between points
for i in range(len(coordinates) - 1):
    start = coordinates[i]
    end = coordinates[i + 1]

    # Draw the line
    line = folium.PolyLine([start, end], color="blue", weight=3, opacity=0.7).add_to(m)

    # Add directional arrow
    plugins.PolyLineTextPath(
        line,
        '➤',  # arrow symbol
        repeat=True,
        offset=7,
        attributes={'fill': 'blue', 'font-weight': 'bold', 'font-size': '16'}
    ).add_to(m)

# Save and show the map
m.save("map_with_arrows.html")
m


In [ ]:
import sys
sys.path.append("..")
from utils.cellspace import CellSpace

In [ ]:
import pickle

porto_cs = pickle.load(open("/home/sagemaker-user/TrajCL/data/porto_20200_cell100_cellspace.pkl","rb"))


In [ ]:
porto_cs

In [ ]:
porto_cs.size()

In [ ]:
lon_lat_list = [[-8.660646, 41.168574],
 [-8.661087, 41.167926],
 [-8.661231, 41.166576],
 [-8.660637, 41.166396],
 [-8.660295, 41.166819],
 [-8.658954, 41.168394],
 [-8.657649, 41.169906],
 [-8.656371, 41.171454],
 [-8.654706, 41.173479],
 [-8.653014, 41.17527],
 [-8.651349, 41.17644],
 [-8.652213, 41.177241],
 [-8.651358, 41.178069],
 [-8.65071, 41.178924],
 [-8.65125, 41.17968],
 [-8.649648, 41.180643],
 [-8.647515, 41.18193],
 [-8.644491, 41.183541],
 [-8.641701, 41.184108],
 [-8.638677, 41.182452],
 [-8.635284, 41.180589],
 [-8.632476, 41.179104],
 [-8.629983, 41.179365],
 [-8.629857, 41.179374],
 [-8.629857, 41.179374],
 [-8.629263, 41.179437],
 [-8.625996, 41.179779],
 [-8.622018, 41.180463],
 [-8.61885, 41.181912],
 [-8.617077, 41.182821],
 [-8.614215, 41.183082],
 [-8.613675, 41.183136],
 [-8.613702, 41.183145],
 [-8.613288, 41.18319],
 [-8.610813, 41.184],
 [-8.607474, 41.184648],
 [-8.604018, 41.183892],
 [-8.601768, 41.183253],
 [-8.601588, 41.182632],
 [-8.601957, 41.181831],
 [-8.601903, 41.181795],
 [-8.601948, 41.181822],
 [-8.601894, 41.181813]]

In [ ]:
porto_cs.x_min, porto_cs.x_max

In [ ]:
from utils.tool_funcs import lonlat2meters

In [ ]:
lonlat2meters(lon_lat_list[0][0],lon_lat_list[0][1])

In [ ]:
point_ids = [porto_cs.get_cellid_by_point(*lonlat2meters(x[0],x[1])) for x in lon_lat_list]

In [ ]:
point_ids

In [ ]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# List available shares.
shares = ddl.list_shares()

# Print shares.
print(shares)

In [ ]:
from ah_databricks_data_loader import DatabricksDataLoader

# Instantiate the DatabricksDataLoader.
ddl = DatabricksDataLoader()

# Load data.
df = ddl.load_as_spark(schema="datascience_scratchpad", table="la_nyc_train_with_time_index")

# Display the loaded DataFrame.
df.count()

In [ ]:
count=0
for row in df.toLocalIterator():
    print(row)
    count+=1
    if count>10:
        break

In [ ]:
row['traj_id']

In [ ]:
splits = df.randomSplit([0.8, 0.1, 0.1], seed=42)  # seed for reproducibility
train_df, val_df, test_df = splits

In [ ]:
train_df.show()

In [ ]:
df_with_index = df.rdd.zipWithIndex().map(lambda x: (x[1], x[0])).toDF(["row_index", "row"])

In [ ]:
df_with_index.filter(df_with_index['row_index']==2).collect()[0]

In [ ]:
next(df.toLocalIterator())

In [ ]:
df.repartition(200).write.parquet("/home/sagemaker-user/TrajCL/data/la_nyc/train", mode="overwrite")

In [ ]:
test = pd.read_parquet("/home/sagemaker-user/TrajCL/data/parquet_files/nyc_v2/part-00000-76372e3b-95b4-498e-9e85-2765482fcef5-c000.snappy.parquet")
test.head()

In [ ]:
len(test)

In [ ]:
import numpy as np
np.stack(test['merc_seq_filtered'].values[0])

In [ ]:
import os
import torch
from torch.utils.data import IterableDataset
import pandas as pd
import numpy as np
class ParquetDataset(IterableDataset):
    def __init__(self, data_dir):
        self.files = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith(".parquet")]

    def __iter__(self):
        for file_path in self.files:
            df = pd.read_parquet(file_path)
            features = df["merc_seq_filtered"].values
            for feature in features:
                yield torch.tensor(np.stack(feature))

In [ ]:
parquet_dataset = ParquetDataset("/home/sagemaker-user/TrajCL/data/parquet_files/nyc/")

In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(parquet_dataset, batch_size=1)

In [ ]:
count = 0
for batch in dl:
    print(batch)
    count+=1
    if count>10:
        break

In [ ]:
l = [[-13163439.650912903,4048092.398836508],[-13161446.041284235,4048306.126552223],[-13158272.155622486,4043076.6409753733],[-13159624.832150962,4040810.9079364412],[-13160802.63689135,4040193.062702784],[-13158150.027009135,4042929.1703618774],[-13158267.034925906,4043092.0348181375],[-13159663.359826725,4039262.957443599],[-13158516.279265795,4043249.4027639213],[-13158274.782762466,4043104.9011289035],[-13158254.010545485,4043086.1864998727],[-13158272.111094687,4043236.751403048],[-13158277.821784565,4043083.7530625863],[-13160550.887862923,4047715.1772090313],[-13164298.7590831,4048060.777160277],[-13164293.994608898,4048092.869597983],[-13164339.457488935,4048215.389297801],[-13162619.916446602,4048008.643955088],[-13158353.496774407,4043341.391084063],[-13158150.027009135,4043015.2405888364],[-13160486.58972504,4040237.9556918466],[-13158680.319667427,4043695.436090043],[-13160159.4217416,4045400.4328289786],[-13160917.084459834,4045195.05647539],[-13158310.238020286,4044011.817762613],[-13158882.520390505,4044559.4392936598],[-13158312.00800019,4044008.980762352],[-13158308.13408191,4044029.068352894],[-13158311.462534685,4044037.9155091126],[-13158949.490196167,4044163.9546188717],[-13161405.354010353,4048318.393488346],[-13163439.650912903,4048092.398836508],[-13163158.546934756,4048472.605607757],[-13163317.600223199,4048258.282937766],[-13163164.313284375,4048482.1018607523]]

In [ ]:
import pickle
import sys
sys.path.append("..")
cellspace = pickle.load(open("/home/sagemaker-user/TrajCL/data/generic_small_v2_cell250_cellspace.pkl", 'rb'))

max_len_meters = min(cellspace.x_max-cellspace.x_min, cellspace.y_max-cellspace.y_min)
def get_min_max_xy(traj):
    min_x = min([p[0] for p in traj])
    max_x = max([p[0] for p in traj])
    min_y = min([p[1] for p in traj])
    max_y = max([p[1] for p in traj])
    return min_x, max_x, min_y, max_y

def calculate_dx_dy(k, target_min_x, target_min_y, min_x, max_x, min_y, max_y):
    dx = target_min_x+ (k - max_x - min_x) / 2
    dy = target_min_y+ (k - max_y - min_y) / 2
    return dx, dy

def transform_traj(traj, dx, dy):
    new_traj = [[p[0] + dx, p[1] + dy] for p in traj]
    return new_traj


min_x, max_x, min_y, max_y = get_min_max_xy(l)
dx, dy = calculate_dx_dy(max_len_meters, cellspace.x_min, cellspace.y_min, min_x, max_x, min_y, max_y)
new_l = transform_traj(l, dx, dy)
new_l

In [ ]:
cellspace.x_min, cellspace.x_max, cellspace.y_min, cellspace.y_max, cellspace.x_max-cellspace.x_min, cellspace.y_max-cellspace.y_min

In [ ]:
def merc2cell2(src, cs):
    # convert and remove consecutive duplicates
    tgt = [ (cs.get_cellid_by_point(*p), p) for p in src]
    # don't execute this if you want to keep the consecutive duplicate points. 
    # tgt = [v for i, v in enumerate(tgt) if i == 0 or v[0] != tgt[i-1][0]]
    tgt, tgt_p = zip(*tgt)
    return tgt, tgt_p

In [ ]:
cellspace_2 = pickle.load(open("/home/sagemaker-user/TrajCL/data/nyc_cell250_cellspace.pkl", 'rb'))

In [ ]:
l_cell, l_merc = merc2cell2(l, cellspace_2)
# old_l_cell, old_l_merc = merc2cell2(l, cellspace_2)

In [ ]:
l_cell

In [ ]:
old_l_cell

In [ ]:
import pickle
import sys
sys.path.append("..")

cellspace_parent = pickle.load(open("/home/sagemaker-user/TrajCL/data/usa_large_cell_cell50000_cellspace.pkl", 'rb'))
cellspace_child = pickle.load(open("/home/sagemaker-user/TrajCL/data/usa_small_cell_cell250_cellspace.pkl", 'rb'))

In [ ]:
from utils.cellspace import HirearchicalCellSpace

hier_cellspace = HirearchicalCellSpace(cellspace_parent, cellspace_child)

In [ ]:
for x,y in l:
    parent, child = hier_cellspace.get_parent_child_cellid(x,y)
    print(f"Point: ({x},{y}) => Parent Cell: {parent}, Child Cell: {child}")